In [1]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import numpy as np



Using TensorFlow backend.


In [2]:
from keras import applications
def vgg16Model(image_size = (224, 224), if_draw_model = False):
    width = image_size[0]
    height = image_size[1]
    
    base_model = applications.VGG16(input_tensor=Input((height, width, 3)), weights='imagenet', include_top=False)

    for layers in base_model.layers:
        layers.trainable = True
        
    for i, layer in enumerate(base_model.layers):
        print(i, layer.name)
        
   # for layer in base_model.layers[25:]:
   #     layer.trainable = True

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.35)(x)
    x = Dense(10, activation='sigmoid')(x)
    model = Model(base_model.input, x)
    
    if if_draw_model:
        SVG(model_to_dot(model).create(prog='dot', format='svg'))
    
    lr = optimizers.SGD(lr=0.000085, momentum=0.9,decay=0.0, nesterov=False)
    lr = optimizers.Adam(lr=1e-3)
    lr = optimizers.Adadelta(lr=0.01)
    model.compile(optimizer=lr,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [3]:
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
from keras.models import *

def restNet_model(image_size = (224, 224), if_draw_model = False):

    width = image_size[0]
    height = image_size[1]
    
    base_model = ResNet50(input_tensor=Input((height, width, 3)), weights='imagenet', include_top=False)

    for layers in base_model.layers:
        layers.trainable = True
        
    for i, layer in enumerate(base_model.layers):
        print(i, layer.name)
        
   # for layer in base_model.layers[25:]:
   #     layer.trainable = True

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(10, activation='sigmoid')(x)
    model = Model(base_model.input, x)
    
    if if_draw_model:
        SVG(model_to_dot(model).create(prog='dot', format='svg'))
    
    lr = optimizers.Adam(lr=1e-4)
   # lr = optimizers.SGD(lr=0.0001, momentum=0.9,decay=0.0, nesterov=False)
    lr = optimizers.Adadelta(lr=0.1)
    model.compile(optimizer= lr,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [4]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input, decode_predictions
import numpy as np

def inceptionV3_model(image_size = (299, 299), if_draw_model = False):

    width = image_size[0]
    height = image_size[1]
    
    base_model = InceptionV3(input_tensor=Input((height, width, 3)), weights='imagenet', include_top=False)

    for layers in base_model.layers:
        layers.trainable = False
        
    for i, layer in enumerate(base_model.layers):
        print(i, layer.name)
        
    for layer in base_model.layers[14:]:
        layer.trainable = True

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(10, activation='sigmoid')(x)
    model = Model(base_model.input, x)
    
    if if_draw_model:
        SVG(model_to_dot(model).create(prog='dot', format='svg'))
    
    lr = optimizers.SGD(lr=0.0001, momentum=0.9,decay=0.0, nesterov=False)
    lr = optimizers.Adam(lr=5e-5)
  #  lr = optimizers.Adadelta(lr=0.007)
    model.compile(optimizer=lr,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [5]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.inception_resnet_v2 import preprocess_input, decode_predictions
import numpy as np

def inception_resnet_v2_model(image_size = (299, 299), if_draw_model = False):

    width = image_size[0]
    height = image_size[1]
    
    base_model = InceptionV3(input_tensor=Input((height, width, 3)), weights='imagenet', include_top=False)

    for layers in base_model.layers:
        layers.trainable = False
        
    for i, layer in enumerate(base_model.layers):
        print(i, layer.name)
        
    for layer in base_model.layers[0:]:
        layer.trainable = True

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(10, activation='sigmoid')(x)
    model = Model(base_model.input, x)
    
    if if_draw_model:
        SVG(model_to_dot(model).create(prog='dot', format='svg'))
    
    lr = optimizers.SGD(lr=0.0005, momentum=0.9,decay=0.0, nesterov=False)
  #  lr = optimizers.Adam(lr=0.1)
    lr = optimizers.Adadelta(lr=0.1)
    model.compile(optimizer=lr,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [2]:
#简单模型
def simple_model(time_len=1):
    ch, row, col = 3, 224, 224  # camera format
    model = Sequential()
    model.add(Lambda(lambda x: x/127.5 - 1.,
  #  model.add(Lambda(lambda x: x,
            input_shape=( row, col, ch),
            output_shape=( row, col,ch)))
    model.add(Convolution2D(3, 3, 3, subsample=(2, 2), border_mode="same"))
    model.add(ELU())
    
    model.add(Flatten())
    model.add(Dropout(.5))
    model.add(ELU())
    model.add(Dense(10))
#  model.add(Lambda(nor_output_1))
    sgd = optimizers.SGD(lr=0.00003, momentum=0.9, nesterov=True)
    model.compile(loss='mean_squared_error', optimizer=sgd)

      

    return model

In [6]:
import os
import shutil
import pandas as pd
import cv2
from sklearn.utils import shuffle
np.random.seed(2017)

def get_driver_list():
    driver_id_list = []
    df = pd.read_csv("driver_imgs_list.csv")
    for i in range(df.shape[0]):
        driver_id = df.loc[i]["subject"]
        is_saved_id = False
        for saved_id in driver_id_list:
            if saved_id == driver_id:
                is_saved_id = True
                break
        
        if is_saved_id == False:
            driver_id_list.append(driver_id)
    return driver_id_list

In [10]:

def prepare_data():
    dirbase = 'imgs/train2/'
    if os.path.exists(dirbase):
        return
    os.mkdir(dirbase)
        
    driver_id_list = get_driver_list()
    
    df = pd.read_csv("driver_imgs_list.csv")  
    sub_dirs = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
    for driver_id in driver_id_list:
        os.mkdir(dirbase + driver_id)
        for sub_dir in sub_dirs:
            os.mkdir(dirbase + driver_id + '/' + sub_dir)

    for i in range(df.shape[0]):
        driver_id = df.loc[i]["subject"]
        class_name = df.loc[i]["classname"]
        img = df.loc[i]["img"]
        add_old = 'imgs/train/' + class_name+'/' + img
        add_new = dirbase + driver_id + '/' + class_name + '/' + img
        #if os.path.exists(add_old):
        shutil.move(add_old, add_new)
        
    os.mkdir('imgs/data')
    shutil.move('imgs/test', 'imgs/data/')
        
prepare_data() 



In [7]:
image_size=(299, 299)
batch_size =64

In [11]:
#训练
from keras.callbacks import *
from keras.models import *
#from keras.applications.vgg19 import preprocess_input
#from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.applications.inception_v3 import preprocess_input, decode_predictions

import copy
def get_file_num(driver_id):
    train_base_add = 'imgs/train2/'
    add_driver = train_base_add + driver_id
    sub_dirs = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
    num = 0
    for sub_dir in sub_dirs:
        add_driver_class = add_driver + '/' + sub_dir
        train_filenames = os.listdir(add_driver_class)
        num += len(train_filenames)
    return num

def train_model(Model, save_file='model.h5'):
    model = Model()
    
    train_datagen = ImageDataGenerator(
 #   shear_range=0.2,
 #   zoom_range=0.2,
    preprocessing_function=preprocess_input,
 #   horizontal_flip=True
    )
    driver_id_list = get_driver_list()
    train_base_add = 'imgs/train2/'
    
    
    patience = 4
    loss_list = []
    
    best_model = 0
    save_loss_callback = LambdaCallback(on_epoch_end=lambda epoch, logs:loss_list.append(logs['val_loss']))
                            
    for epoch in range(25):
        print('step---------------------- {%d}'%(epoch))
        for driver_id in driver_id_list:
            nb_train_samples = get_file_num(driver_id)
            
            train_generator = train_datagen.flow_from_directory(
                train_base_add + driver_id,
                target_size=(image_size[0], image_size[1]),
                batch_size=batch_size,
                class_mode='categorical')
    
            val_generator = train_datagen.flow_from_directory(
                'imgs/val',
                target_size=(image_size[0], image_size[1]),
                batch_size=batch_size,
                class_mode='categorical')
        
            model.fit_generator(
                train_generator,
                steps_per_epoch=nb_train_samples // batch_size,
                epochs=1,
                workers=6,
                callbacks=[save_loss_callback],
                validation_data = val_generator)
            
            if epoch >= 0:
                last_loss = loss_list[-1]
                if_best_loss = True
                for loss in loss_list:
                    if loss < last_loss:
                        if_best_loss = False
                if if_best_loss == True:
                #    best_model =copy.deepcopy(model) 
                #    if epoch >= 6:
                    print('last model loss ------------------------------------------%f'%(last_loss))
                    model.save(save_file)
    
    
    model = load_model(save_file)
    
    return model


In [8]:
from keras.callbacks import *
from keras.models import *
#from keras.applications.vgg16 import preprocess_input
#from keras.applications.resnet50 import preprocess_input, decode_predictions
#from keras.applications.inception_v3 import preprocess_input, decode_predictions
from keras.applications.inception_resnet_v2 import preprocess_input, decode_predictions

from sklearn.preprocessing import OneHotEncoder
from skimage.io import imread, imsave
from scipy.misc import imresize
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

def get_file_num(driver_id):
    train_base_add = 'imgs/train2/'
    add_driver = train_base_add + driver_id
    sub_dirs = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
    num = 0
    for sub_dir in sub_dirs:
        add_driver_class = add_driver + '/' + sub_dir
        train_filenames = os.listdir(add_driver_class)
        num += len(train_filenames)
    return num

def load_image(path):
    img = imread(path)
    img = imresize(img, (image_size[1], image_size[0]))
    return img

def load_driver_imgs(driver_id, pre_add = 'imgs/train2/'):
    base_add = pre_add + driver_id
    sub_dirs = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
    
    img_list = []
    label_list = []
    for i, sub_dir in enumerate(sub_dirs) :
        img_class_add = base_add + '/' + sub_dir
        img_names = os.listdir(img_class_add)
        for img_name in img_names:
            
            img = load_image(img_class_add + '/' + img_name)
            img_list.append(img)
        
            label_list.append(sub_dir)
            
    encoder = LabelEncoder()
    encoder.fit(label_list)
    encoded_Y = encoder.transform(label_list)
    encoded_Y.astype(np.float32)
# convert integers to dummy variables (i.e. one hot encoded)
    label_list = np_utils.to_categorical(encoded_Y)
 
  #  img_list = np.array(img_list)
  #  label_list = np.array(label_list)
    return img_list, label_list

def train_model_2(Model, save_file='model.h5'):
    model = Model()
    
    train_datagen = ImageDataGenerator(
 #   shear_range=0.2,
 #   zoom_range=0.2,
    preprocessing_function=preprocess_input,
 #   horizontal_flip=True
    )
    
    val_datagen = ImageDataGenerator(
 #   shear_range=0.2,
 #   zoom_range=0.2,
    preprocessing_function=preprocess_input,
 #   horizontal_flip=True
    )
    
    driver_id_list = get_driver_list()
    train_base_add = 'imgs/train2/'
    
    
    patience = 5
    loss_list = []
    
    val_driver_list = driver_id_list[0:4]
    train_driver_list = driver_id_list[4:]
   
    x_val = []
    y_val = []
    for driver_id in val_driver_list:
        print(driver_id)
        x, y = load_driver_imgs(driver_id)
        x_val.extend(x)
        y_val.extend(y)
    x_val = np.array(x_val)
    y_val = np.array(y_val)
    val_datagen.fit(x_val)
    
    x_train = []
    y_train = []
    for driver_id in train_driver_list:
        nb_train_samples = get_file_num(driver_id)
            
        x, y = load_driver_imgs(driver_id)
        x_train.extend(x)
        y_train.extend(y)
    
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    
    train_datagen.fit(x_train)
    
    model.fit_generator(
                    train_datagen.flow(x_train, y_train, batch_size=batch_size),
                    shuffle=True,
                    steps_per_epoch=len(x_train) // batch_size,
                    epochs=60,
                    workers=6,
                    callbacks=[EarlyStopping(monitor='val_loss', patience=4, verbose=0)],
                    validation_data = val_datagen.flow(x_val, y_val, batch_size=batch_size)               
                )
 #   model = load_model('model.h5')
    model.save(save_file)
    return model


In [9]:
import pandas as pd
import cv2
def test_model(model, file_name):
    df = pd.read_csv("sample_submission.csv")
    
    test_address = 'imgs_2/data/test'
    test_filenames = os.listdir(test_address)
  #  test_filenames = test_filenames[:10]
    X = np.zeros((len(test_filenames), image_size[0], image_size[1], 3), dtype=np.uint8)

    for i, test_file in enumerate(test_filenames) :
        X[i] = cv2.resize(cv2.imread(test_address + '/' + test_file), image_size)
    
    y_pred = model.predict(X, batch_size=batch_size, verbose=0)
    
    
    for i, fname in enumerate(test_filenames):
      #  print(fname)
        df.set_value(i, 'img', fname)
        sub_dirs = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
        for ii, sb in enumerate(sub_dirs):
            df.set_value(i, sb, y_pred[i][ii])


    df.to_csv(file_name, index=None)
    df.head(10)
    

In [9]:
import pandas as pd
import cv2
#from keras.applications.vgg16 import preprocess_input
#from keras.applications.inception_v3 import preprocess_input, decode_predictions
#from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.applications.inception_resnet_v2 import preprocess_input, decode_predictions

def test_model_2(model, file_name):
    df = pd.read_csv("sample_submission.csv")
    
    test_address = 'imgs/data'

  #  test_address = 'imgs_2/data/test'
    test_filenames = os.listdir(test_address + '/test')
    
    test_datagen = ImageDataGenerator(
 #   shear_range=0.2,
 #   zoom_range=0.2,
    preprocessing_function=preprocess_input,
 #   horizontal_flip=True
    )
    test_generator = test_datagen.flow_from_directory(
        test_address,
        target_size=(image_size[0], image_size[1]),
        shuffle = False,
        class_mode='categorical',
        batch_size=1)

    filenames = test_generator.filenames
    nb_samples = len(filenames)
   
    print(len(test_filenames))
    y_pred = model.predict_generator(test_generator, steps = len(test_filenames))
  #  y_pred = y_pred.clip(min=0.005, max=0.995)
    print(y_pred)
    print(y_pred.shape)
    for i, fname in enumerate(filenames):
     #   print(fname)
        fname_con_list = fname.split('/')
        df.set_value(i, 'img', fname_con_list[-1])
        sub_dirs = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
        for ii, sb in enumerate(sub_dirs):
            df.set_value(i, sb, y_pred[i][ii])


    df.to_csv(file_name, index=None)
    df.head(10)

In [ ]:
model = train_model_2(simple_model, 'model.h5')
test_model_2(model, 'pred.csv')
data = pd.read_csv("pred.csv")

display(data.head())

/anaconda3/envs/dlnd-tf-lab/lib/python3.5/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(3, (3, 3), strides=(2, 2), padding="same")`
  if __name__ == '__main__':


p002


/anaconda3/envs/dlnd-tf-lab/lib/python3.5/site-packages/ipykernel_launcher.py:25: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


p012
p014
p015


In [11]:
model = train_model_2(vgg16Model, save_file='model_vgg.h5')
test_model_2(model, 'pred.csv')
data = pd.read_csv("pred.csv")

display(data.head())

0 input_1
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool
p002


/home/ubuntu/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:25: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


p012
p014
p015
Epoch 1/60
298/298 [==============================] - 337s 1s/step - loss: 2.2850 - acc: 0.1360 - val_loss: 2.1562 - val_acc: 0.1852
Epoch 2/60
298/298 [==============================] - 132s 444ms/step - loss: 1.5027 - acc: 0.4714 - val_loss: 1.2685 - val_acc: 0.6072
Epoch 3/60
298/298 [==============================] - 132s 444ms/step - loss: 0.5340 - acc: 0.8370 - val_loss: 0.7828 - val_acc: 0.7660
Epoch 4/60
298/298 [==============================] - 132s 444ms/step - loss: 0.2375 - acc: 0.9325 - val_loss: 0.7801 - val_acc: 0.7751
Epoch 5/60
298/298 [==============================] - 133s 445ms/step - loss: 0.1479 - acc: 0.9603 - val_loss: 0.6339 - val_acc: 0.8175
Epoch 6/60
298/298 [==============================] - 133s 445ms/step - loss: 0.1052 - acc: 0.9708 - val_loss: 0.7927 - val_acc: 0.7824
Epoch 7/60
298/298 [==============================] - 133s 445ms/step - loss: 0.0771 - acc: 0.9792 - val_loss: 0.7347 - val_acc: 0.8105
Epoch 8/60
298/298 [================

/home/ubuntu/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:39: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/ubuntu/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:42: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,img_1.jpg,1.924421e-15,6.125342e-16,1.710001e-17,1.697531e-15,1.191249e-16,3.789859e-04,1.003671e-18,1.896270e-15,1.539553e-10,1.948141e-12
1,img_10.jpg,4.295882e-11,3.667059e-13,5.056865e-15,3.184777e-15,1.309251e-15,2.812556e-04,5.283374e-17,1.130282e-14,5.617388e-14,1.771881e-13
2,img_100.jpg,6.303615e-08,1.037752e-09,5.265073e-14,1.639867e-12,3.025315e-12,5.463375e-11,1.706329e-14,1.000593e-11,2.385939e-11,7.323917e-09
3,img_1000.jpg,3.337894e-14,1.182375e-13,2.683266e-06,1.154419e-14,3.252749e-15,6.357867e-12,3.802739e-12,4.007042e-16,4.727758e-02,3.481190e-10
4,img_100000.jpg,7.673723e-11,1.541353e-09,2.174207e-16,2.064765e-02,6.863557e-11,3.999730e-11,1.447392e-13,5.463732e-14,3.052532e-10,9.723730e-10


In [ ]:
#lr adam 0.000085,dr 0.5
model = train_model_2(vgg16Model, save_file='model_vgg.h5')
test_model_2(model, 'pred.csv')
data = pd.read_csv("pred.csv")

display(data.head())

0 input_7
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool
p002


/home/ubuntu/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:25: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


p012
p014
p015
Epoch 1/60
298/298 [==============================] - 134s 450ms/step - loss: 2.4099 - acc: 0.1379 - val_loss: 2.2829 - val_acc: 0.1158
Epoch 2/60
298/298 [==============================] - 134s 449ms/step - loss: 1.7246 - acc: 0.3911 - val_loss: 1.6761 - val_acc: 0.4329
Epoch 3/60
298/298 [==============================] - 134s 450ms/step - loss: 0.7855 - acc: 0.7648 - val_loss: 1.2049 - val_acc: 0.6084
Epoch 4/60
298/298 [==============================] - 134s 450ms/step - loss: 0.4179 - acc: 0.8814 - val_loss: 1.0713 - val_acc: 0.6560
Epoch 5/60
298/298 [==============================] - 134s 450ms/step - loss: 0.2759 - acc: 0.9252 - val_loss: 1.0243 - val_acc: 0.6905
Epoch 6/60
298/298 [==============================] - 134s 450ms/step - loss: 0.2151 - acc: 0.9420 - val_loss: 0.9809 - val_acc: 0.7087
Epoch 7/60
298/298 [==============================] - 134s 450ms/step - loss: 0.1585 - acc: 0.9575 - val_loss: 1.1021 - val_acc: 0.7181
Epoch 8/60
298/298 [=============

In [ ]:
#lr adam 0.0001,dr 0.35
model = train_model_2(restNet_model, save_file='model_resnet.h5')
test_model_2(model, 'pred.csv')
data = pd.read_csv("pred.csv")

display(data.head())

94658560/94653016 [==============================] - 1s 0us/step
0 input_1
1 conv1
2 bn_conv1
3 activation_1
4 max_pooling2d_1
5 res2a_branch2a
6 bn2a_branch2a
7 activation_2
8 res2a_branch2b
9 bn2a_branch2b
10 activation_3
11 res2a_branch2c
12 res2a_branch1
13 bn2a_branch2c
14 bn2a_branch1
15 add_1
16 activation_4
17 res2b_branch2a
18 bn2b_branch2a
19 activation_5
20 res2b_branch2b
21 bn2b_branch2b
22 activation_6
23 res2b_branch2c
24 bn2b_branch2c
25 add_2
26 activation_7
27 res2c_branch2a
28 bn2c_branch2a
29 activation_8
30 res2c_branch2b
31 bn2c_branch2b
32 activation_9
33 res2c_branch2c
34 bn2c_branch2c
35 add_3
36 activation_10
37 res3a_branch2a
38 bn3a_branch2a
39 activation_11
40 res3a_branch2b
41 bn3a_branch2b
42 activation_12
43 res3a_branch2c
44 res3a_branch1
45 bn3a_branch2c
46 bn3a_branch1
47 add_4
48 activation_13
49 res3b_branch2a
50 bn3b_branch2a
51 activation_14
52 res3b_branch2b
53 bn3b_branch2b
54 activation_15
55 res3b_branch2c
56 bn3b_branch2c
57 add_5
58 activatio

/home/ubuntu/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:25: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


p012
p014
p015
Epoch 1/60
298/298 [==============================] - 306s 1s/step - loss: 0.2762 - acc: 0.9331 - val_loss: 0.5556 - val_acc: 0.8427
Epoch 2/60
298/298 [==============================] - 104s 348ms/step - loss: 0.0094 - acc: 0.9983 - val_loss: 0.7615 - val_acc: 0.7981
Epoch 3/60
298/298 [==============================] - 104s 348ms/step - loss: 0.0117 - acc: 0.9974 - val_loss: 0.8385 - val_acc: 0.7711
Epoch 4/60
298/298 [==============================] - 104s 349ms/step - loss: 0.0187 - acc: 0.9944 - val_loss: 0.8355 - val_acc: 0.7523
Epoch 5/60
298/298 [==============================] - 104s 348ms/step - loss: 0.0034 - acc: 0.9993 - val_loss: 0.8843 - val_acc: 0.7551
Epoch 6/60
298/298 [==============================] - 104s 348ms/step - loss: 0.0036 - acc: 0.9990 - val_loss: 0.7040 - val_acc: 0.7960
Epoch 7/60
110/298 [==========>...................] - ETA: 1:01 - loss: 9.3277e-04 - acc: 1.0000

In [ ]:
#lr adadelta 0.001,dr 0.5
model = train_model_2(restNet_model, save_file='model_resnet.h5')
test_model_2(model, 'pred.csv')
data = pd.read_csv("pred.csv")

display(data.head())

0 input_2
1 conv1
2 bn_conv1
3 activation_50
4 max_pooling2d_2
5 res2a_branch2a
6 bn2a_branch2a
7 activation_51
8 res2a_branch2b
9 bn2a_branch2b
10 activation_52
11 res2a_branch2c
12 res2a_branch1
13 bn2a_branch2c
14 bn2a_branch1
15 add_17
16 activation_53
17 res2b_branch2a
18 bn2b_branch2a
19 activation_54
20 res2b_branch2b
21 bn2b_branch2b
22 activation_55
23 res2b_branch2c
24 bn2b_branch2c
25 add_18
26 activation_56
27 res2c_branch2a
28 bn2c_branch2a
29 activation_57
30 res2c_branch2b
31 bn2c_branch2b
32 activation_58
33 res2c_branch2c
34 bn2c_branch2c
35 add_19
36 activation_59
37 res3a_branch2a
38 bn3a_branch2a
39 activation_60
40 res3a_branch2b
41 bn3a_branch2b
42 activation_61
43 res3a_branch2c
44 res3a_branch1
45 bn3a_branch2c
46 bn3a_branch1
47 add_20
48 activation_62
49 res3b_branch2a
50 bn3b_branch2a
51 activation_63
52 res3b_branch2b
53 bn3b_branch2b
54 activation_64
55 res3b_branch2c
56 bn3b_branch2c
57 add_21
58 activation_65
59 res3c_branch2a
60 bn3c_branch2a
61 activati

/home/ubuntu/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:25: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


p012
p014
p015
Epoch 1/60
298/298 [==============================] - 109s 365ms/step - loss: 0.2441 - acc: 0.9403 - val_loss: 0.7114 - val_acc: 0.7778
Epoch 2/60
298/298 [==============================] - 104s 350ms/step - loss: 0.0065 - acc: 0.9990 - val_loss: 0.7147 - val_acc: 0.7987
Epoch 3/60
298/298 [==============================] - 104s 351ms/step - loss: 0.0027 - acc: 0.9994 - val_loss: 0.6159 - val_acc: 0.8242
Epoch 4/60
298/298 [==============================] - 104s 351ms/step - loss: 9.3733e-04 - acc: 0.9998 - val_loss: 0.5811 - val_acc: 0.8315
Epoch 5/60
298/298 [==============================] - 104s 351ms/step - loss: 6.2988e-04 - acc: 0.9999 - val_loss: 0.5754 - val_acc: 0.8363
Epoch 6/60
298/298 [==============================] - 104s 351ms/step - loss: 2.7533e-04 - acc: 1.0000 - val_loss: 0.5589 - val_acc: 0.8433
Epoch 7/60
298/298 [==============================] - 104s 351ms/step - loss: 2.3530e-04 - acc: 1.0000 - val_loss: 0.5694 - val_acc: 0.8433
Epoch 8/60
298/29

In [ ]:

#lr adam 0.00005,dr 0.5 from layer11,229,113,51,5,9,14
model = train_model_2(inceptionV3_model, save_file='model_inceptionV3.h5')
test_model_2(model, 'pred.csv')
data = pd.read_csv("pred.csv")

display(data.head())

0 input_1
1 conv2d_1
2 batch_normalization_1
3 activation_1
4 conv2d_2
5 batch_normalization_2
6 activation_2
7 conv2d_3
8 batch_normalization_3
9 activation_3
10 max_pooling2d_1
11 conv2d_4
12 batch_normalization_4
13 activation_4
14 conv2d_5
15 batch_normalization_5
16 activation_5
17 max_pooling2d_2
18 conv2d_9
19 batch_normalization_9
20 activation_9
21 conv2d_7
22 conv2d_10
23 batch_normalization_7
24 batch_normalization_10
25 activation_7
26 activation_10
27 average_pooling2d_1
28 conv2d_6
29 conv2d_8
30 conv2d_11
31 conv2d_12
32 batch_normalization_6
33 batch_normalization_8
34 batch_normalization_11
35 batch_normalization_12
36 activation_6
37 activation_8
38 activation_11
39 activation_12
40 mixed0
41 conv2d_16
42 batch_normalization_16
43 activation_16
44 conv2d_14
45 conv2d_17
46 batch_normalization_14
47 batch_normalization_17
48 activation_14
49 activation_17
50 average_pooling2d_2
51 conv2d_13
52 conv2d_15
53 conv2d_18
54 conv2d_19
55 batch_normalization_13
56 batch_norma

/home/ubuntu/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:25: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


p012
p014
p015
Epoch 1/60
298/298 [==============================] - 315s 1s/step - loss: 0.6125 - acc: 0.8849 - val_loss: 0.9169 - val_acc: 0.7263
Epoch 2/60
298/298 [==============================] - 120s 403ms/step - loss: 0.0276 - acc: 0.9977 - val_loss: 1.1091 - val_acc: 0.6732
Epoch 3/60
298/298 [==============================] - 120s 403ms/step - loss: 0.0133 - acc: 0.9984 - val_loss: 1.0108 - val_acc: 0.7081
Epoch 4/60
298/298 [==============================] - 120s 402ms/step - loss: 0.0069 - acc: 0.9993 - val_loss: 0.9484 - val_acc: 0.7436
Epoch 5/60
 46/298 [===>..........................] - ETA: 1:34 - loss: 0.0033 - acc: 1.0000

In [ ]:

#lr sgd 0.0005,dr 0.5 
model = train_model_2(inception_resnet_v2_model, save_file='model_inceptionRes.h5')
test_model_2(model, 'pred.csv')
data = pd.read_csv("pred.csv")

display(data.head())

0 input_1
1 conv2d_1
2 batch_normalization_1
3 activation_1
4 conv2d_2
5 batch_normalization_2
6 activation_2
7 conv2d_3
8 batch_normalization_3
9 activation_3
10 max_pooling2d_1
11 conv2d_4
12 batch_normalization_4
13 activation_4
14 conv2d_5
15 batch_normalization_5
16 activation_5
17 max_pooling2d_2
18 conv2d_9
19 batch_normalization_9
20 activation_9
21 conv2d_7
22 conv2d_10
23 batch_normalization_7
24 batch_normalization_10
25 activation_7
26 activation_10
27 average_pooling2d_1
28 conv2d_6
29 conv2d_8
30 conv2d_11
31 conv2d_12
32 batch_normalization_6
33 batch_normalization_8
34 batch_normalization_11
35 batch_normalization_12
36 activation_6
37 activation_8
38 activation_11
39 activation_12
40 mixed0
41 conv2d_16
42 batch_normalization_16
43 activation_16
44 conv2d_14
45 conv2d_17
46 batch_normalization_14
47 batch_normalization_17
48 activation_14
49 activation_17
50 average_pooling2d_2
51 conv2d_13
52 conv2d_15
53 conv2d_18
54 conv2d_19
55 batch_normalization_13
56 batch_norma

/home/ubuntu/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:27: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


p012
p014
p015
Epoch 1/60
298/298 [==============================] - 348s 1s/step - loss: 0.3464 - acc: 0.9257 - val_loss: 1.2708 - val_acc: 0.6223
Epoch 2/60
298/298 [==============================] - 141s 474ms/step - loss: 0.0123 - acc: 0.9979 - val_loss: 0.8702 - val_acc: 0.7351
Epoch 3/60
298/298 [==============================] - 141s 474ms/step - loss: 0.0034 - acc: 0.9996 - val_loss: 0.9187 - val_acc: 0.7436
Epoch 4/60
298/298 [==============================] - 141s 474ms/step - loss: 0.0015 - acc: 0.9999 - val_loss: 0.9002 - val_acc: 0.7472
Epoch 5/60
188/298 [=================>............] - ETA: 49s - loss: 8.8382e-04 - acc: 1.0000

In [98]:
#定义CAM可视化方法
import matplotlib.pyplot as plt
import random
#from keras.applications.vgg19 import preprocess_input
#from keras.applications.inception_v3 import preprocess_input, decode_predictions
from keras.applications.resnet50 import preprocess_input, decode_predictions

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
def cam(model):
    print(np.array(model.layers[173]).shape)
    weights = model.layers[173].get_weights()[0]
    model2 = Model(model.input, [model.layers[172].output, model.output])
        
    test_datagen = ImageDataGenerator(
 #   shear_range=0.2,
 #   zoom_range=0.2,
    preprocessing_function=preprocess_input,
 #   horizontal_flip=True
    )
    test_generator = test_datagen.flow_from_directory(
        'imgs_2/data',
        target_size=(image_size[0], image_size[1]),
        shuffle = False,
        class_mode='categorical',
        batch_size=1)

    out, prediction = model2.predict_generator(test_generator, steps = 15)
    print(prediction.shape)
 #   prediction = prediction[0]
    print(out.shape)
 #   out = out[0]

    for i, pre in enumerate(prediction) :
     #   plt.title('steer %.2f' % (prediction))
        cam = np.matmul(out[i], weights) 
        print(cam.shape)
        print(prediction[i].shape)
        cam = (prediction[i] - 0.0)  * cam      
        cam -= cam.min()
        cam /= cam.max()
        cam -= 0.2
    
        print(cam.shape)
        heatmap = cv2.applyColorMap(np.uint8(255*cam), cv2.COLORMAP_JET)
        heatmap[np.where(cam <= 0.2)] = 0
        

        out = cv2.addWeighted(img, 0.8, heatmap, 0.4, 0)
        plt.imshow(out[:,:,::-1])
        plt.show()
    

In [99]:
cam(model)

()
Found 79726 images belonging to 1 classes.
(15, 10)
(15, 7, 7, 2048)
(7, 7, 10)
(10,)
(7, 7, 10)


error: /tmp/build/80754af9/opencv_1512680316562/work/modules/imgproc/src/colormap.cpp:517: error: (-5) cv::ColorMap only supports source images of type CV_8UC1 or CV_8UC3 in function operator()
